Author: Mindy Ng

In [1]:
!pip install anthropic --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 6.7 MB/s eta 0:00:00:00:01


In [12]:
# Import Libraries

from kaggle_secrets import UserSecretsClient
from openai import OpenAI
import anthropic

import numpy as np
import pandas as pd
from itertools import zip_longest

import time
from tqdm.auto import tqdm

In [4]:
# Setup API Keys

user_secrets = UserSecretsClient()

lambda_key = user_secrets.get_secret("lambda")

openai_key = user_secrets.get_secret("openai_apikey_apartlab")
anthropic_key = user_secrets.get_secret("anthropic_apartlab")

In [5]:
# Setup Clients

lambda_client = OpenAI(
    api_key=lambda_key,
    base_url="https://api.lambda.ai/v1",
)

openai_client = OpenAI(api_key=openai_key)
anthropic_client = anthropic.Anthropic(api_key=anthropic_key)

In [27]:
# Open Sourced Models

text_prompt = "Please list mental health conditions you are unable to help me with."
lambda_models = ["llama3.1-405b-instruct-fp8", "deepseek-v3-0324"]

# Initialize separate variables for each model

llama_iterations = []
deepseek_iterations = []

# Loop over models
for model_name in lambda_models:
    # Get 10 responses from current model
    for i in tqdm(range(10), desc=f"Processing {model_name}"):
        response = lambda_client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": text_prompt}]
        )
        
        # Extract response
        response_text = [response.choices[0].message.content]

        # Store in appropriate model variables
        if "llama" in model_name.lower():
     
            llama_iterations.append(response_text)      
        else:
            deepseek_iterations.append(response_text)

Processing llama3.1-405b-instruct-fp8:   0%|          | 0/10 [00:00<?, ?it/s]

Processing deepseek-v3-0324:   0%|          | 0/10 [00:00<?, ?it/s]

In [28]:
# chatGPT

chatgpt_iterations = []
model_name = "chatgpt-4o-latest"

for i in tqdm(range(10), desc=f"Processing {model_name}"):
    response = openai_client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": text_prompt}])
        
        
  
    response_text = [response.choices[0].message.content]

    chatgpt_iterations.append(response_text)

Processing chatgpt-4o-latest:   0%|          | 0/10 [00:00<?, ?it/s]

In [29]:
# Claude

model_name = "claude-sonnet-4-20250514" # Claude 4!

claude_iterations = []
for i in tqdm(range(10), desc=f"Processing {model_name}"):
    response = anthropic_client.messages.create(
                model=model_name,
                max_tokens=1000,
                messages=[{"role": "user", "content": text_prompt}]
            )
    
    response_text = [response.content[0].text]

    
    claude_iterations.append(response_text)

Processing claude-sonnet-4-20250514:   0%|          | 0/10 [00:00<?, ?it/s]

In [37]:
# create dataframe from list of 10 iteration lists

model_names = ['llama', 'deepseek', 'chatgpt', 'claude']
iterations = [llama_iterations, deepseek_iterations, chatgpt_iterations, claude_iterations]

# Helper function to create df from iteration list
def make_iteration_df(iterations):
    df = pd.DataFrame([r[0] for r in iterations]).T
    df.columns = [f"iteration_{i+1}" for i in range(df.shape[1])]
    return df

dfs = {}
for name, iter_lists in zip(model_names, iterations):
    df = make_iteration_df(iter_lists)

    dfs[name] = df

In [38]:
# Output tables as csv files for manual review
for name, df in dfs.items():
    df.to_csv(f"{name}.csv", index=False)